In [42]:
# precisamos extrair os dados desse csv, para isso, irei utilizar do pandas para a leitura desse csv
# é uma das melhores opcoes para se trabalhar com leitura de csvs

import pandas as pd

df = pd.read_json("hf://datasets/infoslack/mistral-7b-arxiv-paper-chunked/train.jsonl", lines=True)

df[["chunk"]].head()

,chunk
0,"Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayr..."
1,automated benchmarks. Our models are released ...
2,GQA significantly accelerates the inference sp...
3,Mistral 7B takes a significant step in balanci...
4,parameters of the architecture are summarized ...


In [43]:
# apos realizadt a leitura, devemos limpar esses dados, ou seja, deixar somente valores
# que possam agregar semanticamente para nosso contexto, removendo numeros, pontuacoes, espacos desnecessarios
# dessa forma é possivel realizar uma melhor analise dos dados
import spacy  # importa a biblioteca spaCy para processamento de linguagem natural

nlp = spacy.load("pt_core_news_sm")  # carrega o modelo de linguagem para português (pequeno)

def preprocess(text):
    if not isinstance(text, str):  # verifica se o input é uma string, senão retorna string vazia
        return ""
    doc = nlp(text.lower())  # converte o texto para minúsculas e processa com spaCy
    # cria uma lista de lemas (forma base das palavras) filtrando tokens que são letras e não são stopwords
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]  
    print(tokens)  # imprime os tokens lematizados para conferência
    return " ".join(tokens)  # retorna os tokens unidos em uma string novamente

# aplica a função preprocess em cada texto da coluna 'resposta_aberta' e cria uma nova coluna 'processed_text'
df["processed_text"] = df["chunk"].apply(preprocess)

# imprime as primeiras linhas das colunas originais e processadas para comparar
df[["chunk", "processed_text"]].head()


    

['mistral', 'albert', 'jiang', 'Alexandre', 'sablayrolles', 'arthur', 'Mensch', 'chri', 'Bamford', 'Devendra', 'Singh', 'chaplot', 'diego', 'Las', 'casa', 'Florian', 'bressand', 'gianna', 'Lengyel', 'guillaume', 'lample', 'lucile', 'saulnier', 'lélio', 'renard', 'lavaud', 'Lachaux', 'pierrer', 'stock', 'teven', 'Le', 'scao', 'thibaut', 'lavril', 'thoma', 'Wang', 'timothée', 'lacroix', 'William', 'el', 'sayed', 'abstract', 'we', 'introduce', 'mistral', 'language', 'Model', 'engineered', 'alto', 'performance', 'and', 'efficiency', 'mistral', 'outperforms', 'The', 'best', 'Open', 'model', 'llama', 'Across', 'All', 'evaluated', 'benchmarks', 'and', 'The', 'best', 'released', 'model', 'llama', 'In', 'reasoning', 'mathematics', 'and', 'coder', 'Generation', 'our', 'model', 'leverages', 'Attention', 'gqa', 'faster', 'inference', 'coupled', 'with', 'sliding', 'Window', 'Attention', 'swa', 'to', 'effectively', 'handle', 'sequence', 'of', 'Arbitrary', 'Length', 'With', 'reduced', 'inferencer', '

,chunk,processed_text
0,"Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayr...",mistral albert jiang Alexandre sablayrolles ar...
1,automated benchmarks. Our models are released ...,automatedr benchmarks our models are released ...
2,GQA significantly accelerates the inference sp...,Gqa significantly accelerate The inference spe...
3,Mistral 7B takes a significant step in balanci...,mistral takes significant step in balancing th...
4,parameters of the architecture are summarized ...,parameter of the architecturar are summarized ...


In [44]:
# iremos utilizar os chunks pois os textos estao muito grandes, dessa forma podemos ter ganho em perfomance e economizacao de tokens
from langchain.text_splitter import CharacterTextSplitter

texto_longo = " ".join(df["processed_text"].tolist())

# com o separator consiguimos separar corretamente esses chunks
splitter = CharacterTextSplitter(chunk_size=70, chunk_overlap=20, separator=" ")

chunks = splitter.split_text(texto_longo)

print(chunks)

['mistral albert jiang Alexandre sablayrolles arthur Mensch chri Bamford', 'Mensch chri Bamford Devendra Singh chaplot diego Las casa Florian', 'Las casa Florian bressand gianna Lengyel guillaume lample lucile', 'lample lucile saulnier lélio renard lavaud Lachaux pierrer stock teven', 'pierrer stock teven Le scao thibaut lavril thoma Wang timothée lacroix', 'timothée lacroix William el sayed abstract we introduce mistral', 'we introduce mistral language Model engineered alto performance and', 'alto performance and efficiency mistral outperforms The best Open', 'The best Open model llama Across All evaluated benchmarks and The best', 'and The best released model llama In reasoning mathematics and coder', 'and coder Generation our model leverages Attention gqa faster', 'Attention gqa faster inference coupled with sliding Window Attention', 'Window Attention swa to effectively handle sequence of Arbitrary', 'of Arbitrary Length With reduced inferencer cost we also providir', 'we also prov

In [45]:
# agora precisamos fazer o embeddamento para que tranformemos em um vetor
# para armazenar em um banco de dados vetorizado

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

print(f"Total chunks:  {len(chunks)}")

embeddings_chunks = model.encode(chunks)

print(f"Gerados {len(embeddings_chunks)} embeddings para chunks, dimensão {embeddings_chunks[0].shape}")

Total chunks:  445
Gerados 445 embeddings para chunks, dimensão (384,)


In [46]:
# agora devemos armazenar em um banco de dados para que consigamos realizar a busca
# semantica

import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings())

collection = client.get_or_create_collection(name="chunk")

ids = [str(i) for i in range(len(chunks))]

metadatas = [{"Source": "chunk", "chunk_index": i} for i in range(len(chunks))]

collection.add(
    ids=ids,
    documents=chunks,
    embeddings=embeddings_chunks,
    metadatas=metadatas
)

In [47]:
# agora ja podemos realizar uma consulta nesse nosso banco de dados vetorizado

query_text = "what is benchmark?"

query_embedding = model.encode([query_text][0])

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances", "metadatas"]
)

print("Top 3 resultados mais similares")

for doc, dist, meta in zip(results["documents"][0], results["distances"][0], results["metadatas"][0]):
    print(f"Distância: {dist:.4f} | Meta: {meta} | Texto: {doc}\n")

Top 3 resultados mais similares
Distância: 17.5281 | Meta: {'Source': 'chunk', 'chunk_index': 35} | Texto: of without sacrificing Performance on related benchmarks mistral

Distância: 20.2576 | Meta: {'Source': 'chunk', 'chunk_index': 164} | Texto: Performance on benchmarks size and efficiency we computed equivalent

Distância: 21.5446 | Meta: {'chunk_index': 177, 'Source': 'chunk'} | Texto: on some benchmarks there Are some difference Between our evaluation

